# DataTables, Indexes, Pandas, and Seaborn

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import seaborn as sns
import numpy as np
import pandas as pd
from glob import glob
sns.set_context("notebook")

## Getting the Data

https://www.ssa.gov/OACT/babynames/index.html

https://www.ssa.gov/data/

We can run terminal/shell commands directly in a notebook! Here's the code to download the dataset (not running since it takes a while):

    !wget https://www.ssa.gov/oact/babynames/state/namesbystate.zip

In [ ]:
# !wget https://www.ssa.gov/oact/babynames/state/namesbystate.zip
!unzip namesbystate.zip

In [ ]:
!ls

In [ ]:
!head CA.TXT

In [ ]:
!wc -l CA.TXT

In [ ]:
# !cat CA.TXT

## Question 1: What was the most popular name in CA last year?

In [ ]:
ca = pd.read_csv('CA.TXT', header=None, names=['State', 'Sex', 'Year', 'Name', 'Count'])
ca.head()

### Slicing

In [ ]:
ca['Count'].head()

In [ ]:
ca[0:3]

In [ ]:
# ca[0]

In [ ]:
ca.iloc[0:3, 0:2]

In [ ]:
ca.loc[0:3, 'State']

In [ ]:
ca.loc[0:5, 'Sex':'Name']

What is the leftmost column?

In [ ]:
emails = ca.head()
emails.index = ['a@gmail.com', 'b@gmail.com', 'c@gmail.com', 'd@gmail.com', 'e@gmail.com']
emails

In [ ]:
emails.loc['b@gmail.com':'d@gmail.com', 'Year':'Name']

In [ ]:
ca.head()

Now access the data for 2016 and sort it by count to find the most popular name in California.

## Question 2: Most popular names in all states for each year?

### Put all DFs together

In [ ]:
# Make sure that filesizes are managable
!ls -alh *.TXT | head

In [ ]:
glob('*.TXT')

In [ ]:
file_names = glob('*.TXT')

baby_names = pd.concat(
    (pd.read_csv(f, names=['State', 'Sex', 'Year', 'Name', 'Count']) for f in file_names)
).reset_index(drop=True)
baby_names.head()

In [ ]:
len(baby_names)

### Group by state and year

In [ ]:
baby_names[
    (baby_names['State'] == 'CA')
    & (baby_names['Year'] == 1995)
    & (baby_names['Sex'] == 'M')
].head()

Show the count for each state.

Group by state then by year. 

### Grouping by multiple columns

Group by state, year and sex.

In [ ]:
def first(series):
    '''Returns the first value in the series.'''
    return series.iloc[0]

most_popular_names = (
    baby_names
    .groupby(['State', 'Year', 'Sex'])
    .agg(first)
)
most_popular_names

This creates a multilevel index. It is quite complex, but just know that you can still slice.

Find an instance when the most popular name was Samuel.

Select the most popular male names for California in the years 1995-2000. [hint: use .loc]

## Question 3: Can I deduce gender from the last letter of a person’s name?

Survey question time!

### Compute last letter of each name

In [ ]:
baby_names.head()

In [ ]:
baby_names['Name'].apply(len).head()

In [ ]:
baby_names['Name'].str.len().head()

In [ ]:
baby_names['Name'].str[-1].head()

To add column to dataframe:

In [ ]:
baby_names['Last letter'] = baby_names['Name'].str[-1]
baby_names.head()

### Group by last letter and sex

### Visualize our result

Use .plot to get some basic plotting functionality:

In [ ]:
# Why is this not good?
letter_counts.plot.barh(figsize=(15, 15))

Reading the docs shows me that pandas will make one set of bars for each column in my table. How do I move each sex into its own column? I have to use pivot:

In [ ]:
# For comparison, the group above:
# letter_counts = (baby_names
#                  .loc[:, ('Sex', 'Count', 'Last letter')]
#                  .groupby(['Last letter', 'Sex'])
#                  .sum())

last_letter_pivot = baby_names.pivot_table(
    index='Last letter', # the rows (turned into index)
    columns='Sex', # the column values
    values='Count', # the field(s) to processed in each group
    aggfunc=sum, # group operation
)
last_letter_pivot.head()

In [ ]:
last_letter_pivot.plot.barh(figsize=(10, 10))

Why is this still not ideal?

- Plotting raw counts
- Not sorted by any order

In [ ]:
totals = last_letter_pivot['F'] + last_letter_pivot['M']

last_letter_props = pd.DataFrame({
    'F': last_letter_pivot['F'] / totals,
    'M': last_letter_pivot['M'] / totals,
}).sort_values('M')
last_letter_props.head()

In [ ]:
last_letter_props.plot.barh(figsize=(10, 10))

What do you notice?